## Extract from ECMWF grib files to Schism NetCDF input

In [ ]:
#!/usr/bin/env python
#
# Meteo to nc for Schism

from Poseidon.utils import *
from Poseidon.meteo import *
from netCDF4 import Dataset
import datetime

inPATH='/home/critechproc/STORAGE/meteo/'
outPATH='/home/critechproc/STORAGE/SCHISM/UVP/'


#define in a dictionary the properties e.g
dic={'lon0':-35., # lat/lon window
     'lon1':43.,
     'lat0':24.5,
     'lat1':76.5}


#creating a time sequence of the runs

start_date = datetime.datetime(2010,2,1,0)

end_date = datetime.datetime(2016,3,1,0)

time_interval = datetime.timedelta(hours=12)

dt=(end_date-start_date).total_seconds()

ndt=dt/time_interval.total_seconds()

ndt=np.int(ndt)+1


date_list = [start_date + time_interval*x for x in range(ndt)]

#creating a sequence of folder from which we read the meteo.

it=1 #initialize index

In [ ]:
date_list

In [ ]:
for date in date_list:

    yyyy=date.year # this is for facilitating the folder format below

    mm=date.month

    dd=date.day

    hh=date.hour

  # set PATH of the database.

    PATH=inPATH+'/{:04d}/{:02d}/{:02d}/'.format(yyyy,mm,dd)

    dpath=glob.glob(PATH+'*{:04d}{:02d}{:02d}.{:02d}.tropical_cyclone.grib'.format(yyyy,mm,dd,hh))

#set meteo forcing

# specify the forecast times ft1,ft2
    ft1 = 0
    ft2 = 11 #that is 0...12(including) hours

# parse the ecmwf grib file.
    m = ecmwf(mpath=dpath[0],ft1=ft1,ft2=ft2,**dic)


# Below the output specifically for Schism but one can adapt appropriately

#from DataFile class create a file
    d = DataFile(outPATH+'sflux_air_1.{:03d}.nc'.format(it), write=True, create=True,format='NETCDF4_CLASSIC')
    
    print outPATH+'sflux_air_1.{:03d}.nc'.format(it)

## add OPTIONAL attributes to the nc file
    info = {'description' : 'Schism forsing',
    'history' :'JRC Ispra European Commission',
    'source' : 'netCDF4 python module'}

    d.info(**info)


    tu = datetime.datetime.strftime(date,'%Y-%m-%d' )
    bd = np.array([yyyy,mm,dd,0])

    time = np.arange(ft1,ft2+1)/24. + hh/24.


    tt=time[:]#.astype(np.float32)
    pp=m.p[:,:,:]#.astype(np.float32)
    uu=m.u[:,:,:]#.astype(np.float32)
    vv=m.v[:,:,:]#.astype(np.float32)
    lats=m.lats#.astype(np.float32)
    lons=m.lons#.astype(np.float32)


    spfh = np.zeros(vv.shape)

#Write variables
    d.write('time',tt, **{'vname':['time'], 'limited':False,'long_name': 'Time', 'standard_name': 'time', 'units': tu,'base_date': bd})
    d.write('lat',lats,  **{'vname':['nx_grid','ny_grid'],'long_name': 'Latitude', 'standard_name': 'latitude', 'units': 'degrees_north'})
    d.write('lon',lons,  **{'vname':['nx_grid','ny_grid'],'long_name': 'Longitude', 'standard_name': 'longitude', 'units': 'degrees_east'})
    d.write('prmsl',pp,  **{'vname':['time','nx_grid','ny_grid'],'long_name': 'Pressure reduced to MSL', 'standard_name': 'air_pressure_at_sea_level', 'units': 'Pa'})
    d.write('uwind',uu, **{'vname':['time','nx_grid','ny_grid'], 'long_name': 'Surface Eastward Air Velocity', 'standard_name': 'eastward_wind', 'units':
'm/s'})
    d.write('vwind',vv,  **{'vname':['time','nx_grid','ny_grid'],'long_name': 'Surface Northward Air Velocity', 'standard_name': 'northward_wind', 'units'
: 'm/s'})
    d.write('spfh',spfh, **{'vname':['time','nx_grid','ny_grid'], 'long_name': 'Surface Specific Humidity (2m AGL)', 'standard_name': 'specific_humidity',
 'units': '1'})
    d.write('stmp',spfh,  **{'vname':['time','nx_grid','ny_grid'],'long_name': 'Surface Temperature', 'standard_name': 'surface temperature', 'units': 'degrees'})


    d.close()

    it+=1

In [ ]:
f = Dataset('/home/critechproc/STORAGE/SCHISM/EUROP/sflux/sflux_air_1.030.nc')

In [ ]:
f.variables.keys()


In [ ]:
f.variables['lon'][0,1]-f.variables['lon'][0,0]

In [ ]:
f.variables['']